## DIA 039: Documentación Automática de la API con Swagger/OpenAPI

En este día, se añadirá documentación automática a la API utilizando Flasgger, una extensión que permite generar y mostrar la documentación en formato Swagger/OpenAPI. Con esta integración, los usuarios podrán acceder a una interfaz interactiva (disponible en la ruta /docs) donde podrán ver la descripción de cada endpoint, los parámetros que acepta, y probar las peticiones directamente desde el navegador. Esta mejora facilita la comprensión de la API y acelera el proceso de desarrollo y mantenimiento.

Código Completo (api.py)
python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
from flasgger import Swagger
import os
import logging
from datetime import datetime
import time
import threading
import json
import random
from functools import wraps

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Configuración de Flasgger para documentación automática
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": "apispec_1",
            "route": "/apispec_1.json",
            "rule_filter": lambda rule: True,  # incluye todas las rutas
            "model_filter": lambda tag: True,  # incluye todos los modelos
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, config=swagger_config)

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            # Se asume que el atributo 'role' existe; para este ejemplo, si no es "admin", se rechaza
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------

@app.route('/login', methods=['POST'])
def login():
    """
    User Login
    ---
    tags:
      - Auth
    parameters:
      - in: body
        name: credentials
        description: User credentials for login.
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: Token de acceso JWT generado.
        schema:
          type: object
          properties:
            access_token:
              type: string
      400:
        description: Datos incompletos.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Para este ejemplo, se asume que las credenciales son correctas.
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: La aplicación está en funcionamiento.
    """
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    """
    Predict Endpoint - v1
    ---
    tags:
      - Prediction v1
    consumes:
      - multipart/form-data
    parameters:
      - in: formData
        name: file
        type: file
        required: true
        description: Imagen para predecir el dígito.
    responses:
      200:
        description: Predicción realizada con éxito.
        schema:
          type: object
          properties:
            prediccion:
              type: integer
            probabilidad:
              type: number
            version:
              type: string
    """
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    """
    Predict Endpoint - v2
    ---
    tags:
      - Prediction v2
    consumes:
      - multipart/form-data
    parameters:
      - in: formData
        name: file
        type: file
        required: true
        description: Imagen para predecir el dígito.
    responses:
      200:
        description: Predicción realizada con mejoras.
        schema:
          type: object
          properties:
            prediccion:
              type: integer
            probabilidad:
              type: number
            version:
              type: string
            mensaje:
              type: string
    """
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Endpoint de A/B Testing para la Predicción
# ---------------------------
ab_blueprint = Blueprint('api_ab', __name__)
@ab_blueprint.route('/predict', methods=['POST'])
@jwt_required()
def predict_ab():
    """
    Predict Endpoint - A/B Testing
    ---
    tags:
      - Prediction AB
    consumes:
      - multipart/form-data
    parameters:
      - in: formData
        name: file
        type: file
        required: true
        description: Imagen para predecir el dígito.
    responses:
      200:
        description: Predicción realizada mediante A/B Testing.
        schema:
          type: object
          properties:
            prediccion:
              type: integer
            probabilidad:
              type: number
            version:
              type: string
            ab_version:
              type: string
    """
    version = random.choice(['v1', 'v2'])
    if version == 'v1':
        response = predict_v1()
    else:
        response = predict_v2()
    data = response.get_json()
    data['ab_version'] = version
    logger.info(f"A/B Testing: Solicitud enrutada a la versión {version}.")
    return jsonify(data), response.status_code

app.register_blueprint(ab_blueprint, url_prefix='/api/ab')

# ---------------------------
# Endpoint de Feedback (simplificado)
# ---------------------------
@app.route('/feedback', methods=['POST'])
@jwt_required()
def submit_feedback():
    """
    Submit Feedback
    ---
    tags:
      - Feedback
    parameters:
      - in: body
        name: feedback
        schema:
          type: object
          required:
            - prediction
            - correct
          properties:
            prediction:
              type: integer
            correct:
              type: boolean
            comment:
              type: string
    responses:
      201:
        description: Feedback registrado exitosamente.
    """
    current_user = get_jwt_identity()
    data = request.get_json()
    prediction = data.get('prediction')
    correct = data.get('correct')
    comment = data.get('comment', "")
    if prediction is None or correct is None:
        logger.warning(f"Feedback fallido: Datos incompletos del usuario '{current_user}'.")
        return jsonify({"msg": "Prediction and correct flag required"}), 400
    feedback = Feedback(
        username=current_user,
        prediction=prediction,
        correct=correct,
        comment=comment
    )
    db.session.add(feedback)
    db.session.commit()
    logger.info(f"Feedback registrado por '{current_user}': {feedback.to_dict()}")
    # Notificar a los administradores (se omite aquí la lógica de SocketIO para simplificar)
    return jsonify({"msg": "Feedback submitted successfully"}), 201

# ---------------------------
# Endpoint de Retraining Basado en Feedback
# ---------------------------
@app.route('/admin/retrain', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_model_endpoint():
    """
    Retrain Model Endpoint
    ---
    tags:
      - Retraining
    responses:
      202:
        description: Proceso de retraining iniciado.
    """
    def retrain_job():
        logger.info("Inicio del retraining del modelo basado en feedback...")
        time.sleep(10)  # Simula el tiempo de retraining
        with open('updated_model.h5', 'w') as f:
            f.write("Modelo actualizado basado en feedback")
        logger.info("Retraining completado. Modelo actualizado.")
    thread = threading.Thread(target=retrain_job)
    thread.start()
    return jsonify({"msg": "Proceso de retraining iniciado"}), 202

# ---------------------------
# Endpoint de Reporte Automatizado
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
@role_required('admin')
def generate_report():
    """
    Generate Report
    ---
    tags:
      - Reports
    responses:
      200:
        description: Reporte generado.
        schema:
          type: object
          properties:
            total_feedback:
              type: integer
            correct_feedback:
              type: integer
            incorrect_feedback:
              type: integer
            accuracy_percentage:
              type: number
    """
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200
    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100
    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    return jsonify(report), 200

# ---------------------------
# Endpoint de Health Check
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: La aplicación está funcionando correctamente.
    """
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Ejecutar la aplicación con soporte WebSocket y Swagger
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explicación del Código
Integración de Swagger:
La integración de Flasgger permite generar documentación automática basada en los docstrings de cada endpoint. Los endpoints incluyen bloques de documentación en formato YAML, que describen los parámetros, respuestas y etiquetas.

Versionado de la API y A/B Testing:
Se mantienen los Blueprints para las versiones v1 y v2 del endpoint /predict, y se añade un nuevo Blueprint para A/B testing, que distribuye aleatoriamente las solicitudes entre las dos versiones.

Endpoints Adicionales:
Se incluyen endpoints para feedback, retraining y generación de reportes, todos protegidos mediante JWT y, en el caso de endpoints críticos de administración, con un decorador de roles.

Ejecución:
La aplicación se ejecuta utilizando Flask-SocketIO para soportar tanto HTTP como WebSocket, y Flasgger genera la documentación accesible en la ruta /docs/.

Resumen en Viñetas del Día 39 (para referencia)
Objetivo:
Implementar documentación automática de la API utilizando Swagger/OpenAPI (con Flasgger).

Integración de Flasgger:

Configuración de Swagger con un bloque de configuración y especificación en la aplicación.
Documentación de endpoints mediante docstrings en formato YAML.
Versionado y A/B Testing:

Los endpoints versionados (v1 y v2) se documentan y se agrupan.
Se añade un endpoint de A/B testing que distribuye solicitudes aleatoriamente entre las versiones.
Beneficios:

Proporciona documentación interactiva y actualizable accesible en /docs/.
Facilita la comprensión y prueba de la API por parte de desarrolladores y usuarios.
Este sistema de documentación automática mejora la transparencia y facilita la colaboración y el mantenimiento de la API.